In [1]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 100

# import yaml

## Localidades

In [2]:
sead_loc = pd.read_csv('../data/geodata/seade_codigo_ibge.csv')
br_mun   = pd.read_csv('../data/geodata/br_municipios_ibge.csv')
cols = ['geocodigo','nome_mesorregiao','nome_microrregiao']
br_mun = br_mun[cols]

In [3]:
url = "http://api-imp.seade.gov.br/v1/localidade"
r = requests.get(url).json()
df_loc = pd.DataFrame.from_records(r['localidades'])
df_loc = df_loc.merge(sead_loc, how = 'left', left_on='nome', right_on='localidade')
df_loc = df_loc.merge(br_mun, on='geocodigo', how='left')
nomes = ['Total do Estado de São Paulo','Região Metropolitana do Vale do Paraíba e Litoral Norte']
mask = ((df_loc['nome_mesorregiao']=='Vale do Paraíba Paulista') | (df_loc['nome'].isin(nomes)))
# mask = (df_loc['nome'].isin(nomes))

df_loc = df_loc[mask]

In [5]:
df_loc.head()

,codigo,nome,nivel,codigo_pai,nome_pai,geocodigo,localidade,nome_mesorregiao,nome_microrregiao
99,1000,Total do Estado de São Paulo,Estado de São Paulo,0,None,NaN,NaN,NaN,NaN
127,25,Aparecida,Municípios,718,Região de Governo de Guaratinguetá,3502507.0,Aparecida,Vale do Paraíba Paulista,Guaratinguetá
135,576,Arapeí,Municípios,714,Região de Governo de Cruzeiro,3503158.0,Arapeí,Vale do Paraíba Paulista,Bananal
140,35,Areias,Municípios,714,Região de Governo de Cruzeiro,3503505.0,Areias,Vale do Paraíba Paulista,Bananal
155,49,Bananal,Municípios,714,Região de Governo de Cruzeiro,3504909.0,Bananal,Vale do Paraíba Paulista,Bananal


## Variaveis

In [6]:
# url =  "http://api-imp.seade.gov.br/v1/variavel?offset=0&limit=100000"
# r = requests.get(url).json()
# df_var = pd.DataFrame.from_records(r['variavel'])

# var_list = list(df_var.keys())

# Get Data

In [7]:
def get_all_cities(locs,anos):
    ### Generate a Clear Dataframe
    df = pd.DataFrame([1])
    df['codigo_localidade']   = 'missing'
    df['localidade']   = 'missing'
    df['nome_localidade_pai']     = 'missing'
    df['codigo_variavel']          = 'missing'
    df['variavel'] = 'missing'
    df['unidade']             = 'missing'
    df['periodo']             = 'missing'
    df['ano']            = 'missing'
    df['loc'] = 'missing'
    df['year']= 'missing'
    df = df.drop(columns=[0])
    df.to_csv("dados_consolidados.csv", index=False, encoding='utf-8')
    
    count = 0
    for localidade in locs:
        print(count,localidade)
        count+=1
        for ano in anos:
            url = "http://api-imp.seade.gov.br/v1/dados/{}/0/{}".format(localidade,ano)
    #           r = requests.get(url).json()
    #             url = "http://api-imp.seade.gov.br/v1/dados/0/2/2010"
            r = requests.get(url).json()
            try:
                df = pd.DataFrame.from_records(r['dados'])
                df['loc'] = localidade
                df['year']= ano

            except:
                df = pd.DataFrame([1])
                df['codigo_localidade']   = 'missing'
                df['localidade']   = 'missing'
                df['nome_localidade_pai']     = 'missing'
                df['codigo_variavel']          = 'missing'
                df['variavel'] = 'missing'
                df['unidade']             = 'missing'
                df['periodo']             = 'missing'
                df['ano']            = 'missing'
                df['loc'] = localidade
                df['year']= ano
                df = df.drop(columns=[0])

    #         df_final = pd.concat([df_final,df], axis=0)
    #         df.to_csv("dados_consolidados_3.csv", index=False, encoding='utf-8')

            df.to_csv("dados_consolidados.csv", index=False, encoding='utf-8', mode='a', header=False)

In [204]:
# var_list
locs = list(df_loc['codigo'].unique())
anos = [i for i in range(2010,2021)]

In [206]:
get_all_cities(locs,anos)

0 1000
1 25
2 576
3 35
4 49
5 85
6 86
7 97
8 628
9 105
10 134
11 136
12 184
13 202
14 204
15 244
16 249
17 263
18 266
19 272
20 317
21 323
22 356
23 380
24 385
25 611
26 419
27 423
28 443
29 460
30 481
31 486
32 495
33 499
34 500
35 507
36 520
37 541
38 548
39 554
40 697
